| Elemento    | Definição |
| -------- | ------- |
| **Estado Inicial**  |   Matriz do jogo da velha vazia   |
| **Estado Objetivo** | Formar uma sequência de três "X" na horizontal, vertical ou diagonal |
| **Função Sucessor** | Jogadores escolhem um elemento não marcado para marcarem com seu respectivo síbolo ("X" ou "O") |
| **Custo de Caminho** | Ganhou o jogo = 1, Perdeu o Jogo = -1, Empatou = 0 |

**Resumo:**
O jogo se incia em uma matriz 3x3 e o objetivo do jogador é formar uma sequência de três símbolos iguais na horizontal, vertical ou diagonal antes que seu oponente possa fazer o mesmo. Cada jogador pode colocar um símbolo por vês da matriz. O jogo termina quando um dos jogadores completar a sequência descrita anteriormente ou a matriz ser completamente preenchida sem um resultado final.

Será usado um algoritmo minimax, sendo que o jogo possui estado deterministico.


In [20]:
import math
import tkinter as tk

def trocar_jogador(player):
    return 'O' if player == 'X' else 'X'

def checar_vencedor(lista):
    for sequencia in lista:
        conjunto_sequencia = set(sequencia)
        if len(conjunto_sequencia) == 1:
            vencedor = conjunto_sequencia.pop()
            if vencedor in ('X', 'O'):
                return vencedor
    return None

def determinar_termino(linhas):
    colunas = list(zip(*linhas))
    diagonal_1 = [linha[i] for i, linha in enumerate(linhas)]
    diagonal_2 = [linha[2-i] for i, linha in enumerate(linhas)]
    direcoes = linhas + list(colunas) + [diagonal_1, diagonal_2]
    vencedor = checar_vencedor(direcoes)
    if vencedor in ('X','O'):
        return vencedor
    
    elementos = [elemento for linha in linhas for elemento in linha]
    if ' ' not in elementos:
        return ' '
    
    return None

def determinar_pontuacao(estado):
    termino = determinar_termino(estado['matriz'])
    if termino == 'O':
        estado['termino'] = True
        estado['pontos'] = 1
    elif termino == 'X':
        estado['termino'] = True
        estado['pontos'] = -1
    elif termino == ' ':
        estado['termino'] = True
        estado['pontos'] = 0
    else:
        estado['termino'] = False
        estado['pontos'] = 0
    return estado

def sucessores(estado, jogador):
    sucessores = []
    for i in range(3):
        for j in range(3):
            if estado['matriz'][i][j] == ' ':
                new_matriz = [row.copy() for row in estado['matriz']]
                new_matriz[i][j] = jogador
                novo_estado = {
                    'prox': trocar_jogador(jogador),
                    'matriz': new_matriz,
                    'pontos': 0,
                    'termino': False
                }
                determinar_pontuacao(novo_estado)
                sucessores.append(novo_estado)
    return sucessores

def max_value(estado, alfa, beta):
    if estado["termino"]:
        return estado['pontos'], estado

    v = float('-inf')
    melhor_estado = None
    for sucessor in sucessores(estado, 'O'): # alterei pois meu oponente é o O
        valor_sucessor, _ = valor(sucessor, alfa, beta)
        if valor_sucessor > v:
            v = valor_sucessor
            melhor_estado = sucessor
        if v >= beta:
            return v, melhor_estado
        alfa = max(alfa, v)
    return v, melhor_estado

def min_value(estado, alfa, beta):
    if estado["termino"]:
        return estado['pontos'], estado

    v = float('inf')
    melhor_estado = None
    for sucessor in sucessores(estado, 'X'):
        valor_sucessor, _ = valor(sucessor, alfa, beta)
        if valor_sucessor < v:
            v = valor_sucessor
            melhor_estado = sucessor
        if v <= alfa:
            return v, melhor_estado
        beta = min(beta, v)
    return v, melhor_estado

def valor(estado, alfa, beta):
    if estado['termino']: 
        return estado['pontos'], estado
    if estado['prox'] == 'O': # alterei pois o meu oponente é o O
        return max_value(estado, alfa, beta)
    if estado['prox'] == 'X': 
        return min_value(estado, alfa, beta)

class JogoDaVelha:
    def __init__(self, root):
        self.root = root
        self.root.title("Jogo da velha")
        self.botoes = [[None for _ in range(3)] for _ in range(3)]
        self.criar_interface()
    
    def criar_interface(self):
        matriz = [
            [' ', ' ', ' '],
            [' ', ' ', ' '],
            [' ', ' ', ' ']
        ]
        estado = {
            'prox': 'O',
            'matriz': matriz,
            'pontos': 0,
            'termino': False
        }
        self.estado = estado
        for i in range(3):
            for j in (range(3)):
                btn = tk.Button(self.root, text="", font=('Arial', 24), width=5, height=2,
                                command=lambda x=i, y=j: self.jogada(x, y))
                btn.grid(row=i, column=j)
                self.botoes[i][j] = btn
        self.botao = tk.Button(self.root, text="Reset", font=('Arial', 14), command=self.resetar_jogo)
        self.botao.grid(row=4, column=0, columnspan=3, pady=10)

    def resetar_jogo(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        self.__init__(self.root)


    def jogada(self, x, y):
        if (self.estado['termino'] or self.estado['matriz'][x][y] != ' '):
            return 
        
        self.botoes[x][y]['text'] = 'X'
        self.estado['matriz'][x][y] = 'X'    

        vencedor = determinar_termino(self.estado['matriz'])

        if (vencedor in ('X', 'O')):
            self.estado['termino'] = True
            self.fim_de_jogo(f"Jogador {self.estado['prox']} venceu!")
            return  
        elif vencedor == ' ':
            self.fim_de_jogo("Empate!")
        
        self.estado['prox'] = 'O'
        self.root.after(500, self.jogada_ia)

    def jogada_ia(self):
        _, novo_estado = max_value(self.estado, -math.inf, math.inf)
        self.estado = novo_estado

        if not novo_estado:
            return

        for i in range(3):
            for j in range(3):
                self.botoes[i][j]['text'] = self.estado['matriz'][i][j]

        vencedor = determinar_termino(self.estado['matriz'])
        if (vencedor):
            self.estado['termino'] = True
            self.fim_de_jogo(f"Jogador {vencedor} venceu!" if vencedor in ('X', 'O') else 'Empate')
        else:
            self.estado['prox'] = 'X'

    def fim_de_jogo(self, mensagem):
        for linha in self.botoes:
            for btn in linha:
                btn.config(state='disabled')
        resultado = tk.Label(self.root, text=mensagem, font=('Arial', 18))
        resultado.grid(row=3, column=0, columnspan=3, pady=10)

if __name__ == "__main__":
    root = tk.Tk()
    jogo = JogoDaVelha(root)
    root.mainloop()
